In [3]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'

# use os.environ function to reference 'SPARK_VERSION' property
os.environ['SPARK_VERSION'] = spark_version
# Install the spark and java components
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
# set the runtime environment variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64" # sets the path to the Java Virtual Machine (Java Development Kit)
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"
# start a spark session by importing and using findspark
import findspark
findspark.init()


Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [4]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [6]:
#add the url to a file to the sparkfiles context
spark.sparkContext.addFile(url)

#make df for file
salesDF = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',')

#show the new df file
salesDF.show(truncate=False)


+------------------------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|id                                  |date               |date_built|price |bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+------------------------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d6-9c31-7398aa8f6089|2022-04-08 00:00:00|2016      |936923|4       |3        |3167       |11733   |2     |1         |76  |
|7530a2d8-1ae3-4517-9f4a-befe060c4353|2021-06-13 00:00:00|2013      |379628|2       |2        |2235       |14384   |1     |0         |23  |
|43de979c-0bf0-4c9f-85ef-96dc27b258d5|2019-04-12 00:00:00|2014      |417866|2       |2        |2127       |10575   |2     |0         |0   |
|b672c137-b88c-48bf-9f18-d0a4ac62fb8b|2019-10-16 00:00:00|2016      |239895|2       |2        |1631       |11149   |2     |0         |0   |
|e0726d4d-d595-4074-

In [7]:
#2. create a temporary view of the dataframe
salesDF.createOrReplaceTempView("home_sales")

In [ ]:
#view the temp data frame
spark.sql("SELECT * FROM home_sales LIMIT 5").show()

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("SELECT YEAR(date) AS YEAR, ROUND(AVG((price),2) As averagePrice, From home_sales WHERE bedrooms = 4").show()


In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("SELECT YEAR(date) AS YEAR, ROUND(AVG((price),2) As averagePrice, From home_sales WHERE bedrooms = 3 bathrooms = 3").show()


In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql(
    "SELECT YEAR(date) AS YEAR, ROUND(AVG((price),2) As averagePrice, From home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_livng = >=2000").show()


In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("SELECT ROUND(AVG((view),2), From home_sales WHERE price = >=350000").show()


print("--- %s seconds ---" % (time.time() - start_time))

--- 7.200241088867188e-05 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.qul("cache table hole_sales")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("SELECT ROUND(AVG((view),2), From home_sales WHERE price = >=350000").show()


print("--- %s seconds ---" % (time.time() - start_time))


--- 5.459785461425781e-05 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
salesDF.write.partitionBy("date_built").parquet("home_sales", mode= "overwrite")

In [ ]:
# 11. Read the parquet formatted data.
salesparquet = spark.read.parquet("home_sales")

In [ ]:
# 12. Create a temporary table for the parquet data.
salesparquet.createorReplaceTempview("salesparquettemporary")

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("SELECT ROUND(AVG((view),2), From salesparquettemporary WHERE price = >=350000").show()


print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("home_sales")